Install the dependencies

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web
import matplotlib.pyplot as plt
plt.style.use("ggplot") # plt.style.available[:] gives list of all available stylesheets
%matplotlib inline
import seaborn as sns
import datetime as dt

import yfinance as yf # as a means to access yahoo finance data

from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.svm import SVC # svm
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score
from sklearn.model_selection import GridSearchCV # c, gamma parameter optimisation

plt.rcParams["font.size"] = 11 # rc refers to run commands
plt.rcParams["figure.facecolor"] = "#000080"
sns.set_style("darkgrid")

import warnings # ignores warnings
warnings.filterwarnings('ignore')

In [ ]:
end = dt.datetime.today()
start = dt.datetime(end.year-2, end.month, end.day)
spx = web.DataReader("^GSPC", "yahoo", start, end) #s&p500
ixic = web.DataReader("^IXIC", "yahoo", start, end) #nasdaq

In [ ]:
# moving average
days = [10,20,50]
for i in days: 
    column = f"MA for {i} days"
    spx[column] = spx["Adj Close"].rolling(window=i).mean()

In [ ]:
spx = pd.DataFrame(spx)
spx.columns

In [ ]:
# bollinger bands
length = 50
num_sd = 2
def boll(closing, length, num_sd):
    """
    returns average, upper band, lower band
    """
    av = closing.rolling(window=length).mean()
    sd = closing.rolling(window=length).std()
    up = av + (num_sd*sd)
    down = av - (num_sd*sd)
    return np.round(down,6), np.round(av,6), np.round(up,6)

In [ ]:
down = f"Lower band"
av = f"Middle band"
up = f"Upper band"
spx[down], spx[av], spx[up] = boll(spx["Adj Close"], length=length, num_sd=num_sd)
spx

In [ ]:
# df = pd.DataFrame(jpm)
# df["Returns"] = (df["Adj Close"]/df["Adj Close"].shift(1))-1

# def direction(x):
#     if x>0:
#         return 1.0
#     else:
#         return 0.0
    
# df["Direction"] = df["Returns"].apply(direction)
# df = df.dropna()

# df = df[["High", "Low", "Open", "Close", "Volume", "Adj Close","Returns", "Direction"]]
# df

In [ ]:
# l = [jpm, gs, ms, bac]
# l_s = ["JPM", "GS", "MS", "BAC"]
# plt.figure(figsize=(15,6))

# for i, j in enumerate(l, 1):
#     plt.subplot(2,2,i)
#     j["Adj Close"].plot()
#     plt.xlabel("Date")
#     plt.ylabel("Adj Close")
#     plt.title(f"Closing price of {l_s[i-1]}")
# plt.tight_layout()

In [ ]:
# data pre-processing
X = df.iloc[:,0:5]
y = df.iloc[:,-1]

# MinMaxScaler
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# train the model
svm = SVC(C=1.0, kernel="linear", gamma="scale")
svm.fit(X_train, y_train)

# prediction
pred = svm.predict(X_test)
pred

# classification report and confusion matrix

print("Classification report:")
print(classification_report(y_test, pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))

print("Accuracy score:")
print(accuracy_score(y_test, pred).round(2))

In [ ]:
# gridsearchcv
param_grid = {"C":[0.1,1,10,100,1000], "gamma":[1,0.1,0.01,0.001]}
svm_grid = GridSearchCV(SVC(), param_grid)
# verbose just shows how much of text output during calculation to display

svm_grid.fit(X_train, y_train)
svm_grid.best_params_

In [ ]:
# # prediction with gridsearchcv (then classification report/confusion matrix)/accuracy score
svm_grid_pred = svm_grid.predict(X_test)

print("Classification report:")
print(classification_report(y_test, svm_grid_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, svm_grid_pred))

print("Accuracy score:")
print(accuracy_score(y_test, svm_grid_pred).round(2))